# Trend Pullback Strategy Walkthrough

This notebook demonstrates the core components of the trend pullback continuation strategy using synthetic data.

## Strategy Overview

The trend pullback strategy identifies high-probability entry points by:
1. **Trend Classification**: Determine if market is trending up, down, or ranging
2. **Pullback Detection**: Wait for counter-trend retracement
3. **Reversal Confirmation**: Detect momentum turn and candlestick patterns
4. **Risk Management**: Calculate position size with ATR-based stops
5. **Execution**: Simulate entry and exit with profit targets and trailing stops

In [ ]:
# Import required libraries
import sys
from pathlib import Path
from datetime import datetime, timedelta, timezone
import random

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from models.core import Candle, TrendState
from indicators.basic import compute_ema, compute_atr, compute_rsi
from strategy.trend_pullback.trend_classifier import classify_trend
from strategy.trend_pullback.pullback_detector import detect_pullback
from strategy.trend_pullback.signal_generator import generate_long_signals
from risk.manager import calculate_position_size
from config.parameters import StrategyParameters

## 1. Generate Synthetic Data

Create a synthetic uptrending market with a pullback scenario.

In [ ]:
def generate_uptrend_with_pullback(n_candles: int = 100, base_price: float = 1.1000) -> list:
    """Generate synthetic uptrend with pullback."""
    candles = []
    current_price = base_price
    start_time = datetime(2025, 1, 1, tzinfo=timezone.utc)
    
    for i in range(n_candles):
        timestamp = start_time + timedelta(minutes=15*i)
        
        # Uptrend for first 60 candles
        if i < 60:
            trend_move = random.uniform(0.00005, 0.0002)
        # Pullback for next 20 candles
        elif i < 80:
            trend_move = random.uniform(-0.0001, -0.00005)
        # Resume uptrend
        else:
            trend_move = random.uniform(0.00005, 0.0002)
        
        current_price += trend_move
        
        # Generate OHLC
        atr_noise = random.uniform(0.00005, 0.0001)
        open_price = current_price
        close_price = current_price + random.uniform(-atr_noise, atr_noise)
        high_price = max(open_price, close_price) + random.uniform(0, atr_noise/2)
        low_price = min(open_price, close_price) - random.uniform(0, atr_noise/2)
        volume = random.uniform(10000, 50000)
        
        candle = Candle(
            timestamp=timestamp,
            open=open_price,
            high=high_price,
            low=low_price,
            close=close_price,
            volume=volume
        )
        candles.append(candle)
    
    return candles

# Generate synthetic dataset
random.seed(42)
candles = generate_uptrend_with_pullback(n_candles=100)
print(f"Generated {len(candles)} synthetic candles")
print(f"First candle: {candles[0].timestamp} - Close: {candles[0].close:.5f}")
print(f"Last candle: {candles[-1].timestamp} - Close: {candles[-1].close:.5f}")

## 2. Trend Classification

Classify the trend at candle 80 (after pullback).

In [ ]:
# Setup parameters
params = StrategyParameters()

# Classify trend at candle 80
trend_result = classify_trend(
    candles=candles[:81],
    ema_fast_period=params.ema_fast_period,
    ema_slow_period=params.ema_slow_period,
    range_threshold=params.range_threshold
)

print(f"Trend State: {trend_result.state}")
print(f"EMA Fast: {trend_result.ema_fast:.5f}")
print(f"EMA Slow: {trend_result.ema_slow:.5f}")
print(f"ATR: {trend_result.atr:.5f}")

## 3. Pullback Detection

Check if a valid pullback has occurred.

In [ ]:
# Detect pullback
pullback_result = detect_pullback(
    candles=candles[:81],
    trend_state=trend_result,
    pullback_threshold=params.pullback_threshold,
    pullback_ema_period=params.pullback_ema_period,
    pullback_max_age=params.pullback_max_age
)

if pullback_result:
    print(f"Pullback detected!")
    print(f"Pullback depth: {pullback_result.pullback_depth:.5f}")
    print(f"Pullback age: {pullback_result.pullback_age_candles} candles")
    print(f"Reversal detected: {pullback_result.reversal_confirmed}")
else:
    print("No pullback detected")

## 4. Signal Generation

Generate long trade signal if all conditions are met.

In [ ]:
# Generate signals for entire dataset
signals = generate_long_signals(
    candles=candles,
    params=params
)

print(f"\nGenerated {len(signals)} long signals")
if signals:
    signal = signals[0]
    print(f"\nFirst Signal Details:")
    print(f"Signal ID: {signal.signal_id}")
    print(f"Timestamp: {signal.timestamp}")
    print(f"Entry Price: {signal.entry_price:.5f}")
    print(f"Stop Price: {signal.stop_price:.5f}")
    print(f"Initial Risk (R): {signal.initial_r:.5f}")
    print(f"Direction: {signal.direction}")

## 5. Position Sizing

Calculate position size based on risk management rules.

In [ ]:
if signals:
    signal = signals[0]
    account_equity = 10000.0  # $10,000 account
    
    position_size = calculate_position_size(
        signal=signal,
        account_equity=account_equity,
        risk_per_trade=params.risk_per_trade,
        max_position_pct=params.max_position_pct
    )
    
    risk_amount = account_equity * params.risk_per_trade
    position_value = position_size * signal.entry_price
    
    print(f"\nPosition Sizing:")
    print(f"Account Equity: ${account_equity:,.2f}")
    print(f"Risk per Trade: {params.risk_per_trade*100:.1f}% (${risk_amount:.2f})")
    print(f"Position Size: {position_size:,.0f} units")
    print(f"Position Value: ${position_value:,.2f}")
    print(f"Leverage: {position_value/account_equity:.2f}x")
    print(f"\nPrice Levels:")
    print(f"Entry: {signal.entry_price:.5f}")
    print(f"Stop: {signal.stop_price:.5f}")
    print(f"Target (3R): {signal.entry_price + 3*signal.initial_r:.5f}")

## 6. Summary

This walkthrough demonstrated:

1. ✅ Synthetic data generation with uptrend and pullback
2. ✅ Trend classification using EMA crossover
3. ✅ Pullback detection with momentum reversal
4. ✅ Trade signal generation with deterministic IDs
5. ✅ Position sizing with ATR-based risk management

## Next Steps

- Run full backtest with `python -m src.cli.run_backtest`
- Analyze metrics (win rate, expectancy, Sharpe ratio)
- Optimize parameters using grid search
- Test on real market data
- Implement short signals for bearish markets